# Getting Started - Matrix Multiplication

In this tutorial, you will learn how to:

1. Install `egg-smol` Python
2. Create a representation for matrices and some simplification rules for them. This will be based off of the [matrix multiplication example](https://github.com/mwillsey/egg-smol/blob/08a6e8f/tests/matrix.egg) in the egg-smol repository. By using our high level wrapper, we can rely on Python's built in static type checker to check the correctness of your representation.
3. Try out using our library in an interactive notebook.

## Install egg-smol Python

First, you will need to have a working Python interpreter. In this tutorial, we will [use `miniconda`](https://docs.conda.io/en/latest/miniconda.html) to create a new Python environment and activate it:

```bash
$ brew install miniconda
$ conda create -n egg-smol-python python=3.11
$ conda activate egg-smol-python
```

Then we want to install `egg-smol` Python. `egg-smol` Python can run on any recent Python version, and is tested on 3.8 - 3.11. To install it, run:

```bash
$ pip install egg-smol
```

To test you have installed it correctly, run:

```bash
$ python -m 'import egg_smol'
```

We also want to install `mypy` for static type checking. This is not required, but it will help us write correct representations. To install it, run:

```bash
$ pip install mypy
```

## Creating an E-Graph

In this tutorial, we will use [VS Code](https://code.visualstudio.com/) to create file, `matrix.py`, to include our egraph
and the simplification rules:

In [2]:
from __future__ import annotations

from egg_smol import *

egraph = EGraph()

## Defining Dimensions

We will start by defining a representation for integers, which we will use to represent
the dimensions of the matrix:

In [3]:
@egraph.class_
class Dim(BaseExpr):
    """
    A dimension of a matix.

    >>> Dim(3) * Dim.named("n")
    Dim(3) * Dim.named("n")
    """
    def __init__(self, value: i64Like) -> None:
        ...

    @classmethod
    def named(cls, name: StringLike) -> Dim:
        ...

    def __mul__(self, other: Dim) -> Dim:
        ...

As you can see, you must wrap any class with the `egraph.class_` to register
it with the egraph and be able to use it like a Python class.

### Testing in a notebook

We can try out this by [creating a new notebook](https://code.visualstudio.com/docs/datascience/jupyter-notebooks#_create-or-open-a-jupyter-notebook) which imports this file:
    
```python
from matrix import *
```

We can then create a new `Dim` object:

In [6]:
x = Dim.named("x")
ten = Dim(10)
res = x * ten * ten
res

(Dim.named("x") * Dim(10)) * Dim(10)

We see that the output is not evaluated, it's just a representation of the computation as well as the type. This is because we haven't defined any simplification rules yet.

We can also try to create a dimension from an invalid type, or use it in an invalid way, we get a type error before we even run the code:

```python
x - ten
```

![Screenshot of VS Code showing a type error](./screenshot-1.png)

## Dimension Replacements

Now we will register some replacements for our dimensions and see how we can interface with egg to get it
to execute them.

In [4]:
a, b, c = vars_("a b c", Dim)
i, j = vars_("i j", i64)
egraph.register(
    rewrite(a * (b * c)).to((a * b) * c),
    rewrite((a * b) * c).to(a * (b * c)),
    rewrite(Dim(i) * Dim(j)).to(Dim(i * j)),
    rewrite(a * b).to(b * a),
)

You might notice that unlike a traditional term rewriting system, we don't specify any order for these rewrites. They will be executed until the graph is fully saturated, meaning that no new terms are created.

We can also see how the type checking can help us. If we try to create a rewrite from a `Dim` to an `i64` we see that we get a type error:

![Screenshot of VS Code showing a type error](./screenshot-2.png)

### Testing
Going back to the notebook, we can test out the that the rewrites are working:
We first have to define our expression, so that it exists in the egraph. We give it a high cost, so that when extracting the best expression, it will avoid the variable itself and instead choose some other equivalent expression:

In [7]:
res_var = egraph.define("res", res, cost=10)

We can then run some number of iterations and extract out the lowest cost expression which is equivalent to our variable:

In [8]:
egraph.run(10)
egraph.extract(res_var)

Dim.named("x") * Dim(100)

We can also extract a number of variants to see all the equivalent expresions, ordered by their cost:

In [9]:
egraph.extract_multiple(res_var, 10)

[Dim(100) * Dim.named("x"),
 Dim.named("x") * Dim(100),
 Dim(10) * (Dim.named("x") * Dim(10)),
 (Dim.named("x") * Dim(10)) * Dim(10),
 res]